# Data Science Bowl Competition - Kaggle 2018

In [ ]:
import os
import pandas as pd
import numpy as np
import scipy.misc
import tensorflow as tf
import time
from matplotlib.pyplot import imshow
from PIL import Image
from skimage.transform import resize
%matplotlib inline

import matplotlib.pyplot as plt
import gc

class Util(object):
    
    @staticmethod
    def read_images(root):
        """
        Read in training images

        Args:
            relative path

        Returns
            train_X: dictionary of image_name -> image
        """

        train_X = {}
        for sub_item_name in os.listdir(root):
            img_root = root + "/" + sub_item_name
            if os.path.isdir(img_root):
                fileName = img_root + "/images/" + sub_item_name + ".png"
                image = scipy.misc.imread(fileName)
                train_X[sub_item_name] = image
        return train_X
    
    def show_image(image):
        resh = np.reshape(image,(image.shape[0],image.shape[1]))
        tout = (np.array(resh) > 0.4).astype(np.uint8)
        imshow(Image.fromarray(tout))
    
    
    def normalize_images(raw_images, height, width, channels):
        """
        Resize images to the given dimension

        Args:
            raw_images: dictiomary of image_name ->image
            height: height of the resized images
            width: width of the resized images
            channels: number of channels in the resized image

        Returns
            List of resized images
        """
        resized = []
        keys = np.sort(list(raw_images.keys()))
        for key in keys:
            resized.append(resize(raw_images[key], (height, width, channels), mode='constant'))
        return np.array(resized)
    
    
    def convert_to_binary_image(image):
        mean = (np.max(image) + np.min(image)) / 2.0
        return image > mean
    
    def show_binary_image(image):
        """
        Display image
        """
        assert image.shape[2] == 1, "image should have shape(,,1)"
        test_mask = image.astype(np.uint8)
        imshow(test_mask.squeeze(), cmap="gray_r")
    
    def decode_run_length(array, encoding):
        """
        Decode run length encoding to the given array

        Args:
            encoding - strinng run length encoding pairs
            array - array to encode to

        """
        encoding_list = encoding.split(" ")
        assert len(encoding_list) % 2 ==0, "Error in encoding"
        for i in range(0,len(encoding_list),2):
            start_index = int(encoding_list[i])-1
            length = int(encoding_list[i+1]) 
            array[start_index:start_index+length] = [1] * length


    def map_run_length_encoding_to_images(run_length_encodings,data_X):
        """
        Maps dictionary of run length encodings to images
        
        Args:
            run_length_encodings: List of run_length_encodings
                 ImageId - correspongind input image in  data_X
                 EncodedPixels - string of pixel encoding pairs
            data_X: originial images from which run length encoding was generated
        Returns
            Dictionary of decoded run length encodings 
        """
        train_Y_orig = {}
        for index,row in run_length_encodings.iterrows(): 
            imageName = row['ImageId']
            encodedPixels = row['EncodedPixels']
            if(imageName not in train_Y_orig):
                w, h, channels = data_X[imageName].shape
                length = w * h
                mask = [0] * length
                train_Y_orig[imageName] = mask

            Util.decode_run_length(train_Y_orig[imageName],encodedPixels)

        for key in train_Y_orig.keys():
            w, h, c = data_X[key].shape
            train_Y_orig[key] = np.reshape(train_Y_orig[key],(w,h,1),order='F')
        return train_Y_orig

class Config(object):
    # Model input image format
    channels = 3
    width = 256
    height = 256
    #batch format
    batch_size = 50
    
    def __init__(self, batch_size):
        self.batch_size = batch_size

##### Config Parameters

In [ ]:
config = Config(batch_size=16)

##### Import and reshape input data

In [ ]:
train_X_orig = Util.read_images("../input/stage1_train")

In [ ]:
train_X = np.array(Util.normalize_images(train_X_orig, config.width, config.height, config.channels))

##### Import predict set

In [ ]:
pred_X_orig = Util.read_images("../input/stage1_test")
pred_X = np.array(Util.normalize_images(pred_X_orig, config.width, config.height, config.channels))

##### Import and decoding run length 

In [ ]:
train_Y_orig = Util.map_run_length_encoding_to_images(pd.read_csv("../input/stage1_train_labels.csv"),train_X_orig)

In [ ]:
train_Y = Util.normalize_images(train_Y_orig, config.width, config.height, 1)

In [ ]:
train_Y = [Util.convert_to_binary_image(image) for image in train_Y]

In [ ]:
t = train_X[0]
print(t.shape)
imshow(t.squeeze())

In [ ]:
t = train_Y[0]
print(t.shape)
imshow(t.squeeze())

In [ ]:
tf.reset_default_graph()

##### Create input placeholder

In [ ]:
#Only the channel size is known
X = tf.placeholder(tf.float32, shape = (None,None,None,config.channels))

In [ ]:
Y = tf.placeholder(tf.float32, shape = (None,None,None,1))

##### Create model

In [ ]:
def forward_propagation(X):

    
    CONV_1_1 = tf.get_variable("CONV_1_1",[3,3,config.channels,16], initializer = tf.contrib.layers.xavier_initializer());
    CONV_1_2 = tf.get_variable("CONV_1_2",[3,3,16,16], initializer = tf.contrib.layers.xavier_initializer());
     
    CONV_2_1 = tf.get_variable("CONV_2_1",[3,3,16,32], initializer = tf.contrib.layers.xavier_initializer());
    CONV_2_2 = tf.get_variable("CONV_2_2",[3,3,32,32], initializer = tf.contrib.layers.xavier_initializer());
    
    CONV_3_1 = tf.get_variable("CONV_3_1",[3,3,32,64], initializer = tf.contrib.layers.xavier_initializer());
    CONV_3_2 = tf.get_variable("CONV_3_2",[3,3,64,64], initializer = tf.contrib.layers.xavier_initializer());
    
    CONV_4_1 = tf.get_variable("CONV_4_1",[3,3,64,128], initializer = tf.contrib.layers.xavier_initializer());
    CONV_4_2 = tf.get_variable("CONV_4_2",[3,3,128,128], initializer = tf.contrib.layers.xavier_initializer());
    
    CONV_5_1 = tf.get_variable("CONV_5_1",[3,3,128,256], initializer = tf.contrib.layers.xavier_initializer());
    CONV_5_2 = tf.get_variable("CONV_5_2",[3,3,256,256], initializer = tf.contrib.layers.xavier_initializer());
    
    CONV_6_1 = tf.get_variable("CONV_6_1",[2,2,128,256], initializer = tf.contrib.layers.xavier_initializer());
    CONV_6_2 = tf.get_variable("CONV_6_2",[3,3,256,128], initializer = tf.contrib.layers.xavier_initializer());
    CONV_6_3 = tf.get_variable("CONV_6_3",[3,3,128,128], initializer = tf.contrib.layers.xavier_initializer());
    
    CONV_7_1 = tf.get_variable("CONV_7_1",[2,2,64,128], initializer = tf.contrib.layers.xavier_initializer());
    CONV_7_2 = tf.get_variable("CONV_7_2",[3,3,128,64], initializer = tf.contrib.layers.xavier_initializer());
    CONV_7_3 = tf.get_variable("CONV_7_3",[3,3,64,64], initializer = tf.contrib.layers.xavier_initializer());
    
    CONV_8_1 = tf.get_variable("CONV_8_1",[2,2,32,64], initializer = tf.contrib.layers.xavier_initializer());
    CONV_8_2 = tf.get_variable("CONV_8_2",[3,3,64,32], initializer = tf.contrib.layers.xavier_initializer());
    CONV_8_3 = tf.get_variable("CONV_8_3",[3,3,32,32], initializer = tf.contrib.layers.xavier_initializer());
    
    CONV_9_1 = tf.get_variable("CONV_9_1",[2,2,16,32], initializer = tf.contrib.layers.xavier_initializer());
    CONV_9_2 = tf.get_variable("CONV_9_2",[3,3,32,16], initializer = tf.contrib.layers.xavier_initializer());
    CONV_9_3 = tf.get_variable("CONV_9_3",[3,3,16,16], initializer = tf.contrib.layers.xavier_initializer());
    
    CONV_OUT = tf.get_variable("CONV_OUT",[1,1,16,1], initializer = tf.contrib.layers.xavier_initializer());
      
    Z1 = tf.nn.relu(tf.nn.conv2d(X,CONV_1_1, strides=[1,1,1,1], padding='SAME'))
    D1 = tf.nn.dropout(Z1,keep_prob = 0.9)
    V1 = tf.nn.relu(tf.nn.conv2d(D1,CONV_1_2, strides=[1,1,1,1], padding='VALID'))
    A1 = tf.nn.max_pool(V1, ksize = [1, 127, 127, 1], strides = [1,1,1,1], padding = 'VALID')
    
    tf.summary.histogram('A1', A1)
    
    Z2 = tf.nn.relu(tf.nn.conv2d(A1, CONV_2_1, strides=[1,1,1,1], padding='SAME'))
    D2 = tf.nn.dropout(Z2, keep_prob = 0.9)
    V2 = tf.nn.relu(tf.nn.conv2d(D2, CONV_2_2, strides=[1,1,1,1], padding='VALID'))
    A2 = tf.nn.max_pool(V2, ksize = [1, 63, 63, 1], strides = [1,1,1,1], padding = 'VALID')
    
    tf.summary.histogram('A2', A2)
    
    Z3 = tf.nn.relu(tf.nn.conv2d(A2, CONV_3_1, strides=[1,1,1,1], padding='SAME'))
    D3 = tf.nn.dropout(Z3, keep_prob = 0.8)
    V3 = tf.nn.relu(tf.nn.conv2d(D3, CONV_3_2, strides=[1,1,1,1], padding='VALID'))
    A3 = tf.nn.max_pool(V3, ksize = [1, 31, 31, 1], strides = [1,1,1,1], padding = 'VALID')
    
    tf.summary.histogram('A3', A3)
    
    Z4 = tf.nn.relu(tf.nn.conv2d(A3, CONV_4_1, strides=[1,1,1,1], padding='SAME'))
    D4 = tf.nn.dropout(Z4, keep_prob = 0.8)
    V4 = tf.nn.relu(tf.nn.conv2d(D4, CONV_4_2, strides=[1,1,1,1], padding='VALID'))
    A4 = tf.nn.max_pool(V4, ksize = [1, 15, 15, 1], strides = [1,1,1,1], padding = 'VALID')

    tf.summary.histogram('A4', A4)
    
    Z5 = tf.nn.relu(tf.nn.conv2d(A4, CONV_5_1, strides=[1,1,1,1], padding='SAME'))
    D5 = tf.nn.dropout(Z5, keep_prob = 0.8)
    Z5 = tf.nn.relu(tf.nn.conv2d(D5, CONV_5_2, strides=[1,1,1,1], padding='SAME'))
   
    tf.summary.histogram('m_Z5', Z5)

    batch_size = tf.shape(Z5)[0]    
    # top of piramid, or bottom of valley
    Z6 = tf.nn.conv2d_transpose(Z5, CONV_6_1, [batch_size,32,32,128], strides = [1,2,2,1], padding='VALID')
    Z6 = tf.concat([Z6,Z4],3)
    Z6 = tf.nn.relu(tf.nn.conv2d(Z6, CONV_6_2, strides=[1,1,1,1], padding='SAME'))
    D6 = tf.nn.dropout(Z6, keep_prob = 0.8)
    Z6 = tf.nn.relu(tf.nn.conv2d(D6, CONV_6_3, strides=[1,1,1,1], padding='SAME'))
    
    tf.summary.histogram('deconv-Z6', Z6)
    
    Z7 = tf.nn.conv2d_transpose(Z6, CONV_7_1, [batch_size,64,64,64], strides = [1,2,2,1], padding='VALID')
    Z7 = tf.concat([Z7,Z3],3)
    Z7 = tf.nn.relu(tf.nn.conv2d(Z7, CONV_7_2, strides=[1,1,1,1], padding='SAME'))
    D7 = tf.nn.dropout(Z7, keep_prob = 0.8)
    Z7 = tf.nn.relu(tf.nn.conv2d(D7, CONV_7_3, strides=[1,1,1,1], padding='SAME'))
    
    tf.summary.histogram('deconv_Z7', Z7)
    
    Z8 = tf.nn.conv2d_transpose(Z7, CONV_8_1, [batch_size,128,128,32], strides = [1,2,2,1], padding='VALID')
    Z8 = tf.concat([Z8,Z2],3)
    Z8 = tf.nn.relu(tf.nn.conv2d(Z8, CONV_8_2, strides=[1,1,1,1], padding='SAME'))
    D8 = tf.nn.dropout(Z8, keep_prob = 0.9)
    Z8 = tf.nn.relu(tf.nn.conv2d(D8, CONV_8_3, strides=[1,1,1,1], padding='SAME'))
    
    tf.summary.histogram('deconv_Z8', Z8)
    
    Z9 = tf.nn.conv2d_transpose(Z8, CONV_9_1, [batch_size,256,256,16], strides = [1,2,2,1], padding='VALID')
    Z9 = tf.concat([Z9,Z1],3)
    Z9 = tf.nn.relu(tf.nn.conv2d(Z9, CONV_9_2, strides=[1,1,1,1], padding='SAME'))
    D9 = tf.nn.dropout(Z9, keep_prob = 0.9)
    Z9 = tf.nn.relu(tf.nn.conv2d(D9, CONV_9_3, strides=[1,1,1,1], padding='SAME'))
    
    tf.summary.histogram('deconv_Z6', Z9)
    
    output = tf.nn.sigmoid(tf.nn.conv2d(Z9, CONV_OUT, strides=[1,1,1,1], padding='SAME'))
    tf.summary.histogram('activations', output)
    
    return output

In [ ]:
def group_list(data_list, group_size):
    """
    Yields groups from l sized group_size
    Args:
        data_list: list
        group_size: size of group
    
    Returns
    yield
    """
    for i in range(0, len(data_list), group_size):
        yield data_list[i:i+group_size]

In [ ]:
predictions = forward_propagation(X)

In [ ]:
loss = tf.keras.losses.binary_crossentropy(Y,predictions)
cost = tf.reduce_mean(loss)
tf.summary.scalar('cost', cost)

##### Intersection over union score

In [ ]:
prediction_mask = tf.to_int64(predictions > 0.5)
iou_score, update_op = tf.metrics.mean_iou(Y, prediction_mask, 2)
with tf.control_dependencies([update_op]):
    iou_score = tf.identity(iou_score)
tf.summary.scalar('intersection_overunion', iou_score)

##### Stats

In [ ]:
merged_summary = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('train')
test_writer = tf.summary.FileWriter('test')

In [ ]:
def model(train_X, train_Y, test_X, test_Y, epochs, batch_size = 50, learning_rate = 0.005, save = -1, restore= -1):    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    init = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    saver = tf.train.Saver()
    sess = tf.Session();
    if(True):
        if(restore!=-1):        
            saver.restore(sess,"saved_model/model_" + str(restore))
        else:
            sess.run(init);
            sess.run(init_l)
            
        
        for epoch in range(epochs):
            start_time = time.time()
            train_X_batches = group_list(train_X,batch_size)
            train_Y_batches = group_list(train_Y,batch_size)
            t_number_of_batches = len(train_X) // batch_size
            
            avg_cost_value = 0.
            
            avg_iou = 0.
            
            for index in range(0,t_number_of_batches):                
                train_row_X = next(train_X_batches)
                
                train_row_Y = next(train_Y_batches)
                
                summary,opt, cost_value, iou_score_training = sess.run([merged_summary,optimizer, cost, iou_score],feed_dict={X: train_row_X, Y: train_row_Y})
                train_writer.add_summary(summary, epoch)
                avg_cost_value += cost_value / t_number_of_batches
                avg_iou += iou_score_training / t_number_of_batches
                
            summary,test_cost_value, t_iou_score = sess.run([merged_summary,cost,iou_score],feed_dict={X: test_X, Y: test_Y})
            duration = time.time() - start_time
            test_writer.add_summary(summary, epoch)
            print("Epoch " '%03d' % (epoch+1), 
                  ": training cost = ", avg_cost_value,
                  " IOU score =  ", avg_iou ,
                  " test cost = ", test_cost_value,
                  " IOU score", t_iou_score,
                  " : in ", duration, "s")
                    
        return sess

In [ ]:
sess = model(train_X[:512,:,:,:],
             train_Y[:512],
             train_X[512:,:,:,:],
             train_Y[512:],
             epochs = 50, batch_size = 16)

In [ ]:
pred_Y = sess.run([predictions],feed_dict={X: pred_X})

In [ ]:
f, arr = plt.subplots(3,2)
arr[0][0].imshow(pred_X[0])
arr[0][1].imshow(Util.convert_to_binary_image(pred_Y[0][0]).squeeze())
arr[1][0].imshow(pred_X[1])
arr[1][1].imshow(Util.convert_to_binary_image(pred_Y[0][1]).squeeze())
arr[2][0].imshow(pred_X[2])
arr[2][1].imshow(Util.convert_to_binary_image(pred_Y[0][2]).squeeze())